In [4]:
import cv2
import time
import numpy as np
from skimage import transform
import imutils
from imutils.video import VideoStream
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow import expand_dims
import os

In [5]:
# Load the mask detector model
mask_net = load_model('face_mask_detector_Threeclasses (1).model')

# Load the face detector model
prototxtPath = r'C:\Users\mvcc1\Downloads\TelegramDesktop\caffemodel\deploy.prototxt'
weightsPath = r'C:\Users\mvcc1\Downloads\TelegramDesktop\caffemodel\res10_300x300_ssd_iter_140000.caffemodel'

face_net = cv2.dnn.readNetFromCaffe(prototxtPath, weightsPath)

font = cv2.FONT_HERSHEY_SIMPLEX

# Initialize the webcam
webcam = VideoStream(src=0).start()
time.sleep(2.0)

while True:
    frame = webcam.read()
    frame = imutils.resize(frame, width=600)
    
    faces = []
    locs = [] # Locations
    
    # Grab the dimensions of a frame and then construct a blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224,224), (104.0, 177.0, 123.0))
    # Pass the blob through the face network to get the face detections
    face_net.setInput(blob)
    face_detections = face_net.forward()
    #print(face_detections)
    # Loop over the face detections
    for i in range(0, face_detections.shape[2]):
        # Extract the confidence associated with the face detections
        confidence = face_detections[0, 0, i, 2]
        #print(confidence)
        # Filter out weak detections to ensure the confidence is greater than the minimum
        if confidence > 0.5:
            # Compute the (x, y) coordinates of the bounding box for the object
            box = face_detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype(np.int)
            # Making sure the bounding boxes fall within the dimensions of the frame
            (x, y) = (max(0, x), max(0, y))
            (x1, y1) = (min(w-1, x1), min(h-1, y1))
            try:
                # Slice the frame, convert to RGB and preprocess
                face = frame[y:y1, x:x1]
                face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                face = cv2.resize(face, (224, 224))
                face = img_to_array(face)
                face = preprocess_input(face)
                # Expand to fit the frame
                face = expand_dims(face, axis=0)
                # Append to their respective lists
                faces.append(face)
                locs.append((x, y, x1, y1))
                # Make the predictions('incorrect_mask' 'with_mask' 'without_mask') 
                preds = mask_net.predict(faces) # Making predictions on the frame
                labels = np.argmax(preds)
                #print(labels)
            except:
                pass
    
    for (box, preds) in zip(locs, preds):
        # Unpack the bounding box and predictions
        (x, y, x1, y1) = box
        (wrongMask, Mask, noMask) = preds
       
        if labels == 1:
            cv2.rectangle(frame, (x, y), (x1, y1), (0, 255, 15), 2)
            cv2.putText(frame, 'Mask', (x, y-8), font, 0.5, (0, 255, 15), 2)
        
        elif labels == 0:
            cv2.rectangle(frame, (x, y), (x1, y1), (15, 50, 100), 2)
            cv2.putText(frame, 'Incorrect Mask', (x, y-8), font, 0.5, (15, 50, 100), 2)
            
        else:
            cv2.rectangle(frame, (x, y), (x1, y1), (0, 0, 255), 2)
            cv2.putText(frame, 'No Mask', (x, y-8), font, 0.5, (0, 0, 255), 2)
        
    cv2.imshow('Live!', frame)
    key = cv2.waitKey(5)
    if key == 27:
        break

cv2.destroyAllWindows()
webcam.stop()

OSError: SavedModel file does not exist at: face_mask_detector_Threeclasses (1).model\{saved_model.pbtxt|saved_model.pb}